# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


In [ ]:
pwd

In [ ]:
# !pip install openpyxl
# !pip install selenium
# !pip install webdriver-manager
# !pip install html5lib
# !pip install lxml

In [ ]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# First, open the url and set the timeout to 4 seconds
service = Service(executable_path="chromedriver-mac64/chromedriver")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")

url = "https://infogob.jne.gob.pe/Eleccion"
driver.get( url )
time.sleep(4)

# Second, choose the type of process, in this case 'presidencial'
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()
time.sleep(3)

# Third, get all the elments in list of election years
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
lista_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')
opciones_lista_eleccion = [ x for x in lista_eleccion.find_elements(By.TAG_NAME, "div") ]
dict_opciones_lista_eleccion = { option.text : option for option in opciones_lista_eleccion }

# Fourth, create the data frame where we will save results
df_resultados = pd.DataFrame(columns=['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS'])

# Fifth, iterate for all the elements in the dictionary created based on list of election years
for elemento_txt in list(dict_opciones_lista_eleccion.keys())[1:]:
    elemento = dict_opciones_lista_eleccion[elemento_txt]
    elemento.click() # click on the second element, the first is "SELECCION" so we don't need it

    # click on the button to see results
    boton_ver_datos = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.ID, 'btnVerDatos'))
    ).click()
    time.sleep(5)
    
    # choose the tab 'CANDIDATOS Y RESULTADOS'
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]').click()
    time.sleep(3)

    # found and get the table with results
    ubi_tabla = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]')
    tabla_html = ubi_tabla.get_attribute('innerHTML')
    tabla_resultados = pd.read_html(tabla_html)[0]
    # create a column with the information of the year of election
    tabla_resultados['ELECCIONES'] = elemento_txt
    # keep columns we need
    columnas_deseadas = ['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']
    tabla_resultados = tabla_resultados[columnas_deseadas]